# Import Library

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers

# Mempersiapkan Dataset kita

In [2]:
data = pd.read_csv('Automobile.csv') # Load dataset kita

data = data.dropna() # Menghapus row dengan value yang hilang


# Memilih fitur dan target
X = data[['cylinders']]  # Fitur cylinders
y = data['mpg']  # Target mpg

print(X.head())
print(y.head())

   cylinders
0          8
1          8
2          8
3          8
4          8
0    18.0
1    15.0
2    18.0
3    16.0
4    17.0
Name: mpg, dtype: float64


# Membagi dataset kita menjadi data pelatihan dan data validasi

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

# Normalisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Menampilkan Setiap Ukuran
print(f'Ukuran X_train: {X_train.shape}')
print(f'Ukuran X_val: {X_val.shape}')
print(f'Ukuran y_train: {y_train.shape}')
print(f'Ukuran y_val: {y_val.shape}')

# Menampilkan 5 baris pertama
print("5 baris pertama dari X_train:")
print(X_train[:5])
print("5 baris pertama dari y_train:")
print(y_train[:5])

Ukuran X_train: (235, 1)
Ukuran X_val: (157, 1)
Ukuran y_train: (235,)
Ukuran y_val: (157,)
5 baris pertama dari X_train:
[[ 1.39533735]
 [-0.9138477 ]
 [-0.9138477 ]
 [ 1.39533735]
 [-0.9138477 ]]
5 baris pertama dari y_train:
212    16.5
346    32.3
325    44.3
90     12.0
246    32.8
Name: mpg, dtype: float64


# Mengonversi y menjadi kategori disini mpg > 20 sebagai efficient dan lainnya not efficient

In [4]:
y_train = (y_train > 20).astype(int)
y_val = (y_val > 20).astype(int)

# Membangun Model Deep Learning
### Kita akan menggunakan Sequential dengan 2 Dense layer untuk membuat model regresi sederhana

In [5]:
# Membangun model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Menggunakan sigmoid untuk binary classification
])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,241 (8.75 KB)

 Trainable params: 2,241 (8.75 KB)

 Non-trainable params: 0 (0.00 B)

# Mengompilasi model dengan optimizer adam dan loss function mean_squared_error

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Buat Callback

In [7]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Train model dengan data train dan memvalidasinya dengan data validasi

In [8]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    verbose=1,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 410ms/step - accuracy: 0.6258 - loss: 0.6690 - val_accuracy: 0.8917 - val_loss: 0.6158
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8769 - loss: 0.6040 - val_accuracy: 0.8917 - val_loss: 0.5709
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8863 - loss: 0.5532 - val_accuracy: 0.8917 - val_loss: 0.5289
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8812 - loss: 0.5173 - val_accuracy: 0.8917 - val_loss: 0.4917
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8624 - loss: 0.4866 - val_accuracy: 0.8917 - val_loss: 0.4562
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8836 - loss: 0.4429 - val_accuracy: 0.8917 - val_loss: 0.4230
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8845 - loss: 0.4032 - val_accuracy: 0.8917 - val_loss: 0.3933
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8425 - loss: 0.4011 - val_accuracy: 0.8917 - val_loss: 0.3684

# Evaluate model menggunakan data testing

In [9]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)

print(f'Training accuracy:{train_accuracy *100:.2f}')
print(f'Validation accuracy: {val_accuracy *100:.2f}')

Training accuracy:87.66
Validation accuracy: 89.17
